In [1]:
## importing libraries
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

In [2]:
## reading the files and loading them into dataframes.
train = pd.read_csv('C:/Users/Blenz/Desktop/Standard_Bank/Train.csv')
test= pd.read_csv('C:/Users/Blenz/Desktop/Standard_Bank/Test.csv')
sample = pd.read_csv('C:/Users/Blenz/Desktop/Standard_Bank/sample_submission.csv')
mask = pd.read_csv('C:/Users/Blenz/Desktop/Standard_Bank/unlinked_masked_final.csv')
variabs = pd.read_csv('C:/Users/Blenz/Desktop/Standard_Bank/VariableDefinitions.csv')

In [3]:
## Transform dates types from 'object' to 'datetime'
train.TransactionStartTime=pd.to_datetime(train.TransactionStartTime)
test.TransactionStartTime=pd.to_datetime(test.TransactionStartTime)
train.IssuedDateLoan=pd.to_datetime(train.IssuedDateLoan)
test.IssuedDateLoan=pd.to_datetime(test.IssuedDateLoan)
train.PaidOnDate=pd.to_datetime(train.PaidOnDate)
train.DueDate=pd.to_datetime(train.DueDate)

In [4]:
train.corr()

,Value,Amount,CountryCode,TransactionStatus,AmountLoan,IsFinalPayBack,IsThirdPartyConfirmed,IsDefaulted
Value,1.000000,-0.999995,NaN,-0.008597,0.554949,-0.399866,-0.086357,0.333220
Amount,-0.999995,1.000000,NaN,0.008536,-0.554698,0.400056,0.086438,-0.333401
CountryCode,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TransactionStatus,-0.008597,0.008536,NaN,1.000000,NaN,NaN,NaN,NaN
AmountLoan,0.554949,-0.554698,NaN,NaN,1.000000,-0.045880,-0.195072,0.031049
IsFinalPayBack,-0.399866,0.400056,NaN,NaN,-0.045880,1.000000,-0.069143,-0.594954
IsThirdPartyConfirmed,-0.086357,0.086438,NaN,NaN,-0.195072,-0.069143,1.000000,0.017547
IsDefaulted,0.333220,-0.333401,NaN,NaN,0.031049,-0.594954,0.017547,1.000000


In [5]:
train['mean_loan_cus']=train['CustomerId'].map(train.groupby('CustomerId').agg(('mean','std','min','max')).AmountLoan['mean'])
train['max_loan_cus']=train['CustomerId'].map(train.groupby('CustomerId').agg(('mean','std','min','max')).AmountLoan['max'])
train['min_loan_cus']=train['CustomerId'].map(train.groupby('CustomerId').agg(('mean','std','min','max')).AmountLoan['min'])
train['std_loan_cus']=train['CustomerId'].map(train.groupby('CustomerId').agg(('mean','std','min','max')).AmountLoan['std'])
test['mean_loan_cus']=test['CustomerId'].map(train.groupby('CustomerId').agg(('mean','std','min','max')).AmountLoan['mean'])
test['max_loan_cus']=test['CustomerId'].map(train.groupby('CustomerId').agg(('mean','std','min','max')).AmountLoan['max'])
test['min_loan_cus']=test['CustomerId'].map(train.groupby('CustomerId').agg(('mean','std','min','max')).AmountLoan['min'])
test['std_loan_cus']=test['CustomerId'].map(train.groupby('CustomerId').agg(('mean','std','min','max')).AmountLoan['std'])

In [6]:
## creating variables to transfer the information contained in the rows of the same transaction.
train['Number_Of_Split_Payments'] = 0 ## this is a count on the number of payments on the same loan. It will take a 0 for singled-rowed transactions, 1+ for multi-row transacs.
#train['Sum_Diff_Time_Payments'] = 0 ## I'm thinking of summing the delays between all payments made on a loan. It will take 0 for loans paid in a single time, 1+ for multiple payments on the same loan.
test['Number_Of_Split_Payments']=0
#test['Sum_Diff_Time_Payments']=0

In [7]:
## creating the feature : number of split payments on a loan.
train['Number_Of_Split_Payments']=train['TransactionId'].map(train.groupby('TransactionId').agg('count')['Number_Of_Split_Payments'])
test['Number_Of_Split_Payments']=test['TransactionId'].map(test.groupby('TransactionId').agg('count')['Number_Of_Split_Payments'])

In [8]:
train.drop(train[(train.TransactionId=='TransactionId_703')|((train.TransactionId=='TransactionId_927'))].index,axis=0,inplace=True)

In [9]:
## Lets drop the duplicate rows with the same transaction ID and keep the last one. (as in with the latest payment installment )
train.drop_duplicates(subset=['TransactionId'],keep='last',inplace=True)
test.drop_duplicates(subset=['TransactionId'],keep='last',inplace=True)

In [10]:
train.drop(['CountryCode','Currency','CurrencyCode','SubscriptionId','ProviderId','ChannelId'],axis=1,inplace=True)
test.drop(['CountryCode','CurrencyCode','SubscriptionId','ProviderId','ChannelId'],axis=1,inplace=True)

### Feature engineering

In [11]:
train['Count_Rejected_Loans'] = train['CustomerId'].map(train[train.TransactionStatus==0].groupby('CustomerId').LoanId.size())
test['Count_Rejected_Loans'] = test['CustomerId'].map(train[train.TransactionStatus==0].groupby('CustomerId').LoanId.size())
## then we should impute the columns of customers that were not found in the rejected list with 0 as in they have never been rejected.
train.Count_Rejected_Loans.fillna(value=0,inplace=True)
test.Count_Rejected_Loans.fillna(value=0,inplace=True)

In [12]:
## group train/test together to perform cumulative count
all_data=pd.concat((train,test)).copy()
## Initialize and compute values for the new feature
all_data['Cumulative_Reject']=0
all_data.loc[all_data.TransactionStatus==0,'Cumulative_Reject'] = all_data[all_data.TransactionStatus==0].groupby('CustomerId').cumcount()
## Separate all_data into train and test
train1=all_data[:len(train)]
test1=all_data[len(train):]
train['Cumulative_Reject']=0
test['Cumulative_Reject']=0
train['Cumulative_Reject']=train1['Cumulative_Reject']
test['Cumulative_Reject']=test1['Cumulative_Reject']

C:\Users\Blenz\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [13]:
purchasestats=train[train.TransactionStatus==0].groupby('CustomerId').Value.agg(('mean','std','min','max'))
train['prchs_mean']=train['CustomerId'].map(purchasestats['mean'])
train['prchs_std']=train['CustomerId'].map(purchasestats['std'])
train['prchs_max']=train['CustomerId'].map(purchasestats['max'])
train['prchs_min']=train['CustomerId'].map(purchasestats['min'])
test['prchs_mean']=test['CustomerId'].map(purchasestats['mean'])
test['prchs_std']=test['CustomerId'].map(purchasestats['std'])
test['prchs_max']=test['CustomerId'].map(purchasestats['max'])
test['prchs_min']=test['CustomerId'].map(purchasestats['min'])

In [14]:
valuegroups=mask.groupby('CustomerId').Value.agg(('mean','std','min','max','count'))
train['mean_cus_transac']=train['CustomerId'].map(valuegroups['mean'])
train['std_cus_transac']=train['CustomerId'].map(valuegroups['std'])
train['min_cus_transac']=train['CustomerId'].map(valuegroups['min'])
train['max_cus_transac']=train['CustomerId'].map(valuegroups['max'])
train['count_cus_transac']=train['CustomerId'].map(valuegroups['count'])
test['mean_cus_transac']=test['CustomerId'].map(valuegroups['mean'])
test['std_cus_transac']=test['CustomerId'].map(valuegroups['std'])
test['min_cus_transac']=test['CustomerId'].map(valuegroups['min'])
test['max_cus_transac']=test['CustomerId'].map(valuegroups['max'])
test['count_cus_transac']=test['CustomerId'].map(valuegroups['count'])

In [15]:
train['Day_Of_Week']= train.TransactionStartTime.dt.weekday
test['Day_Of_Week'] =test.TransactionStartTime.dt.weekday
train['Day_in_month']=train.TransactionStartTime.dt.day
test['Day_in_month']=test.TransactionStartTime.dt.day

In [16]:
from datetime import date
datemin = date(2018,9,21)
datemax= date(2019,7,17)
(datemax-datemin).days
datesinc=pd.DataFrame(columns=['date','inc_value'])
datesinc.loc[0,'inc_value']=1
datesinc.loc[0,'date']=datemin
from datetime import timedelta
for i in range(2,301):
    datesinc.loc[i-1,'inc_value']=i
    datesinc.loc[i-1,'date']=datemin + timedelta(days=i-1)
train['inc_value_date']=train.TransactionStartTime.dt.date.map(datesinc.set_index('date').inc_value)
test['inc_value_date']=test.TransactionStartTime.dt.date.map(datesinc.set_index('date').inc_value)

In [17]:
train.inc_value_date = train.inc_value_date.astype(np.int64)
test.inc_value_date = test.inc_value_date.astype(np.int64)

In [18]:
aa=train[(train.TransactionStatus==1)&(train.TransactionStartTime<train.DueDate)].groupby('CustomerId').agg(('count','mean','std','min','max')).Value
#train['number_transac_before_due']=train['CustomerId'].map(aa['count'])
train['before_due_mean'] = train['CustomerId'].map(aa['mean'])
train['before_due_std'] = train['CustomerId'].map(aa['std'])
train['before_due_min'] = train['CustomerId'].map(aa['min'])
train['before_due_max'] = train['CustomerId'].map(aa['max'])
test['before_due_mean'] = test['CustomerId'].map(aa['mean'])
test['before_due_std'] = test['CustomerId'].map(aa['std'])
test['before_due_min'] = test['CustomerId'].map(aa['min'])
test['before_due_max'] = test['CustomerId'].map(aa['max'])

In [21]:
train[train.IsDefaulted==1].describe()

,Value,Amount,TransactionStatus,AmountLoan,IsFinalPayBack,IsThirdPartyConfirmed,IsDefaulted,mean_loan_cus,max_loan_cus,min_loan_cus,...,min_cus_transac,max_cus_transac,count_cus_transac,Day_Of_Week,Day_in_month,inc_value_date,before_due_mean,before_due_std,before_due_min,before_due_max
count,71.000000,71.000000,71.0,71.000000,71.000000,71.000000,71.0,71.000000,71.000000,71.000000,...,32.000000,32.000000,32.000000,71.000000,71.000000,71.000000,71.000000,14.000000,71.000000,71.000000
mean,166982.943662,-166943.647887,1.0,20857.464789,0.042254,0.985915,1.0,19592.459958,23329.267606,17026.633803,...,4860.156250,16843.125000,21.312500,2.084507,15.183099,146.084507,164268.767392,19996.569919,161373.225352,167344.915493
std,118712.571933,118764.447454,0.0,27400.273355,0.202599,0.118678,0.0,25583.863224,26476.284703,25920.723835,...,8512.163639,11212.081168,34.221704,1.636603,9.051140,28.643995,117253.454881,63672.473880,119503.505741,118216.399362
min,1500.000000,-342000.000000,1.0,500.000000,0.000000,0.000000,1.0,500.000000,500.000000,500.000000,...,225.000000,900.000000,1.000000,0.000000,1.000000,36.000000,1166.666667,0.000000,500.000000,1500.000000
25%,14000.000000,-192000.000000,1.0,4060.000000,0.000000,1.000000,1.0,3713.694444,10000.000000,1320.000000,...,500.000000,10000.000000,2.000000,0.000000,6.000000,147.500000,13647.500000,1749.652709,11000.000000,14000.000000
50%,192000.000000,-192000.000000,1.0,16000.000000,0.000000,1.000000,1.0,16000.000000,17000.000000,16000.000000,...,900.000000,16000.000000,11.000000,2.000000,15.000000,158.000000,192000.000000,3092.212441,192000.000000,192000.000000
75%,192000.000000,-13500.000000,1.0,25000.000000,0.000000,1.000000,1.0,20000.000000,30500.000000,19000.000000,...,2750.000000,20450.000000,20.000000,4.000000,25.000000,165.000000,192000.000000,4713.192877,192000.000000,192000.000000
max,342000.000000,-1500.000000,1.0,192000.000000,1.000000,1.000000,1.0,192000.000000,192000.000000,192000.000000,...,37000.000000,41500.000000,178.000000,6.000000,30.000000,168.000000,342000.000000,241123.412385,342000.000000,342000.000000


In [22]:
train=train[train.IsDefaulted.notnull()]

In [24]:
train.loc[:,'new_customer']=0
test.loc[:,'new_customer']=0
train.loc[train.mean_cus_transac.isnull(),'new_customer']=1
test.loc[test.mean_cus_transac.isnull(),'new_customer']=1

In [26]:
features = [#'CustomerId', #'TransactionStartTime', 
            'Value', #'Amount',
       #'TransactionId', 
            #'BatchId', 
              #'ProductId',
    'mean_cus_transac',
    #'std_cus_transac', 
    'min_cus_transac', 
    'max_cus_transac', 
       #'ProductCategory', #'TransactionStatus', 
            #'IssuedDateLoan',
    #'mean_value_on_missed',
       #'LoanId', , 'LoanApplicationId', 'ThirdPartyId',
       #'Number_Of_Split_Payments', 
        #    'Count_Rejected_Loans', 
           #'Cumulative_Reject',
       #'prchs_mean', 'prchs_std',
    'prchs_max', 
    'prchs_min', #'month',
         #   'InvestorId',
            #'Day_Of_Week',
    'Day_in_month',#'mean_loan_cus',
    #'max_loan_cus',
    #'min_loan_cus',
    'new_customer',
    #'inc_value_date'
    #'before_due_mean'#, 'before_due_std',
       'before_due_min', 'before_due_max', 
            'count_cus_transac'#,'Cnt_missed_payment'
]

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
X = train[features]
X_test = test[features]
y=train.IsDefaulted.copy()

In [30]:
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.2,stratify=y,random_state=10)

In [62]:
xgb = XGBClassifier(max_depth=4,colsample_bytree=0.6,min_child_weight=10,learning_rate=0.25,n_estimators=100,objective = "binary:logistic",random_state=26)
xgb.fit(X_train,y_train)
val_pred=xgb.predict_proba(X_val)[:,1]
print(roc_auc_score(y_val,val_pred))

0.9953917050691244


In [64]:
xgb.fit(X,y)
test_pred=xgb.predict_proba(X_test)[:,1]

In [65]:
sample_submission = pd.DataFrame(columns=['TransactionId','IsDefaulted'])
sample_submission['TransactionId'] = test['TransactionId']
sample_submission['IsDefaulted'] = test_pred

In [66]:
sample_submission.to_csv('C:/Users/Blenz/Desktop/Non.csv',index=False)